## Install Necessary Libraries ##

In [ ]:
## ONCE DETECTRON2 IS INSTALLED, PLEASE COPY THE CONTENTS FROM THE SUBFOLDER DETECTRON2 FOLDER TO THE MAIN DETECTRON2 FOLDER!!!

# Upgrade pip
%pip install --upgrade pip

# Install OpenCV for image processing
%pip install opencv-python

# Install numpy at the desired version
%pip install numpy==1.26.1

# Install PyTorch and related packages (adjust versions/URLs as needed for your platform)
%pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.0+cu117 --index-url https://download.pytorch.org/whl/cu117

# Install gitpython and cython if not already installed
%pip install gitpython cython

# Install dependencies, but pin fvcore and pycocotools to compatible versions:
%pip install cython

# Install fvcore.
%pip install fvcore==0.1.5.post20221221

# Install pycocotools
%pip install pycocotools==2.0.8

# ---- Additional installations ----
%pip install docutils==0.19
%pip install sphinx==7
%pip install recommonmark==0.6.0
%pip install sphinx_rtd_theme
%pip install termcolor
%pip install yacs
%pip install tabulate
%pip install cloudpickle
%pip install future
# (The following two lines with CPU wheels are for Linux CP37; comment them out if not needed)
# %pip install https://download.pytorch.org/whl/cpu/torch-1.8.1%2Bcpu-cp37-cp37m-linux_x86_64.whl
# %pip install https://download.pytorch.org/whl/cpu/torchvision-0.9.1%2Bcpu-cp37-cp37m-linux_x86_64.whl
%pip install "omegaconf>=2.1.0.dev24"
%pip install "hydra-core>=1.1.0.dev5"
%pip install scipy
%pip install timm
# ---------------------------------

# Clone the Detectron2 repository and install it in editable mode
import os
if not os.path.exists('detectron2'):
    !git clone https://github.com/facebookresearch/detectron2.git

%cd detectron2
%pip install -e .
%cd ..

# Install TensorFlow and Keras for deep learning models
%pip install tensorflow

# Install pickleshare to fix the IPython warning
%pip install pickleshare

# Install additional required libraries
%pip install numpy matplotlib pandas scikit-learn tqdm Pillow

# Install geospatial libraries
%pip install geopandas rasterio shapely

# Install any other utility libraries if needed
%pip install jsonschema pyyaml

# Install IPython kernel to ensure compatibility
%pip install ipykernel

# After installations, you may need to restart the kernel
print("All libraries installed. Please restart the kernel to ensure all packages are loaded.")


## Import Necessary Libraries ##

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import copy
import random
import glob

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from yacs.config import CfgNode as CN
import detectron2.data.transforms as T
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
print(np.__version__)


## Expanding Dataset (Do not run unless you need to make dataset bigger, the changes are permanent) ##

In [ ]:
# # Define the train folder and its subfolders
# train_folder = r"C:\Users\itagl\Downloads\AerialImageDataset\train"
# images_folder = os.path.join(train_folder, "images")
# gt_folder = os.path.join(train_folder, "gt")

# # Get a list of all TIF images in the images folder
# image_paths = glob.glob(os.path.join(images_folder, "*.tif"))
# print(f"Found {len(image_paths)} images in {images_folder}.")

# def rotate_image(img, angle):
#     """Rotate image by given angle (90, 180, 270 degrees)."""
#     if angle == 90:
#         return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
#     elif angle == 180:
#         return cv2.rotate(img, cv2.ROTATE_180)
#     elif angle == 270:
#         return cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
#     else:
#         raise ValueError("Unsupported rotation angle: " + str(angle))

# for img_path in image_paths:
#     base = os.path.splitext(os.path.basename(img_path))[0]  # e.g. 'austin1'
#     ext = os.path.splitext(img_path)[1]  # e.g. '.tif'
#     # Assume corresponding mask has the same base name in the gt folder
#     mask_path = os.path.join(gt_folder, base + ext)
    
#     # Read the original image and mask
#     img = cv2.imread(img_path)
#     mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)  # keep mask as-is
#     if img is None or mask is None:
#         print(f"Skipping {base}{ext}: cannot load image or mask.")
#         continue
    
#     # 1. Rotate 90
#     img_r90 = rotate_image(img, 90)
#     mask_r90 = rotate_image(mask, 90)
    
#     # 2. Rotate 180
#     img_r180 = rotate_image(img, 180)
#     mask_r180 = rotate_image(mask, 180)
    
#     # 3. Rotate 270
#     img_r270 = rotate_image(img, 270)
#     mask_r270 = rotate_image(mask, 270)
    
#     # 4. Mirror (flip horizontally)
#     img_m = cv2.flip(img, 1)
#     mask_m = cv2.flip(mask, 1)
    
#     # 5. Rotate 90 then mirror
#     img_r90m = cv2.flip(img_r90, 1)
#     mask_r90m = cv2.flip(mask_r90, 1)
    
#     # 6. Rotate 180 then mirror
#     img_r180m = cv2.flip(img_r180, 1)
#     mask_r180m = cv2.flip(mask_r180, 1)
    
#     # 7. Rotate 270 then mirror
#     img_r270m = cv2.flip(img_r270, 1)
#     mask_r270m = cv2.flip(mask_r270, 1)
    
#     # Save new images in the images folder
#     cv2.imwrite(os.path.join(images_folder, f"{base}r90{ext}"), img_r90)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r180{ext}"), img_r180)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r270{ext}"), img_r270)
#     cv2.imwrite(os.path.join(images_folder, f"{base}m{ext}"), img_m)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r90m{ext}"), img_r90m)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r180m{ext}"), img_r180m)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r270m{ext}"), img_r270m)
    
#     # Save new masks in the gt folder
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r90{ext}"), mask_r90)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r180{ext}"), mask_r180)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r270{ext}"), mask_r270)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}m{ext}"), mask_m)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r90m{ext}"), mask_r90m)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r180m{ext}"), mask_r180m)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r270m{ext}"), mask_r270m)
    
#     print(f"Created 7 copies for {base}{ext}")

# print("Done creating rotated and mirrored copies.")

## Implement get_inria_dicts() ##

In [ ]:
def get_inria_dicts_split(img_dir, mask_dir, split="train", train_ratio=0.8):
    """
    Returns a list of dataset dictionaries for Detectron2.
    Splits the dataset into a training set (80% by default)
    and a validation set (20% by default) based on the 'split' parameter.
    
    Assumes each image in img_dir has a corresponding mask in mask_dir
    with the same filename.
    
    Parameters:
      - img_dir: Directory containing the images.
      - mask_dir: Directory containing the corresponding masks.
      - split: "train" or "val" to specify the subset.
      - train_ratio: Proportion of images for training.
    
    Returns:
      - List of dictionaries with image paths and annotations.
    """
    # Get list of image filenames (filter for common image extensions)
    image_files = [f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png", ".jpeg", ".tif", ".tiff"))]
    
    # Shuffle the list for randomness (set seed if reproducibility is needed)
    random.shuffle(image_files)
    
    # Calculate the split index
    split_index = int(len(image_files) * train_ratio)
    
    # Select files based on split
    if split == "train":
        selected_files = image_files[:split_index]
    else:  # "val"
        selected_files = image_files[split_index:]
    
    dataset_dicts = []
    for idx, image_file in enumerate(selected_files):
        record = {}
        img_path = os.path.join(img_dir, image_file)
        base_name = os.path.splitext(image_file)[0]
        mask_path = os.path.join(mask_dir, image_file)  # same filename for mask
        
        # Read the image to get its dimensions
        img = cv2.imread(img_path)
        if img is None:
            continue
        height, width = img.shape[:2]
        
        record["file_name"] = img_path
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        
        # Read the corresponding mask in grayscale
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue
        
        # Threshold the mask to ensure it's binary
        _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objs = []
        for cnt in contours:
            if cv2.contourArea(cnt) < 10:
                continue
            poly = cnt.flatten().tolist()
            if len(poly) < 6:
                continue
            x, y, w, h = cv2.boundingRect(cnt)
            objs.append({
                "bbox": [x, y, w, h],
                "bbox_mode": BoxMode.XYWH_ABS,
                "segmentation": [poly],
                "category_id": 0
            })
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts


## Register the dataset for Detectron2 ##

In [ ]:
# Register the dataset with Detectron2
# Define the paths to your image and mask folders
train_img_dir = r"C:\Users\itagl\Downloads\AerialImageDataset\train\images"
train_mask_dir = r"C:\Users\itagl\Downloads\AerialImageDataset\train\gt"

# Register training subset (80% of the image-mask pairs)
DatasetCatalog.register("inria_train", lambda: get_inria_dicts_split(train_img_dir, train_mask_dir, split="train"))
MetadataCatalog.get("inria_train").set(thing_classes=["rooftop"])

# Register validation subset (20% of the image-mask pairs)
DatasetCatalog.register("inria_val", lambda: get_inria_dicts_split(train_img_dir, train_mask_dir, split="val"))
MetadataCatalog.get("inria_val").set(thing_classes=["rooftop"])

print("Registered 'inria_train' and 'inria_val' datasets.")


## Visualize sample data to ensure it's loaded correctly ##

In [ ]:
# Visualize a few samples from the dataset
dataset_dicts = DatasetCatalog.get("inria_train")
num_samples = 3

for d in random.sample(dataset_dicts, num_samples):
    img = cv2.imread(d["file_name"])
    if img is None:
        continue
    visualizer = Visualizer(img[:, :, ::-1], MetadataCatalog.get("inria_train"), scale=1.0)
    out = visualizer.draw_dataset_dict(d)
    plt.figure(figsize=(10, 10))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.title("Sample Data Visualization")
    plt.axis("off")
    plt.show()


## Configure and train the Mask R-CNN model ##

In [ ]:
# Create configuration
cfg = get_cfg()
# Load a Mask R-CNN config from the model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))

# Specify the training and validation datasets
cfg.DATASETS.TRAIN = ("inria_train",)
cfg.DATASETS.TEST = ("inria_val",)  # used for evaluation during training

# DataLoader and augmentation settings
cfg.DATALOADER.NUM_WORKERS = 12   # adjust based on your CPU capabilities

# Load pretrained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")

# Solver settings (adjust iterations and learning rate as needed)
cfg.SOLVER.IMS_PER_BATCH = 6    # images per batch
cfg.SOLVER.BASE_LR = 0.00025    # learning rate
cfg.SOLVER.MAX_ITER = 15000      # number of iterations, adjust based on convergence
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  # number of proposals per image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only one class: "rooftop"

# Optional: Enable built-in random cropping to 4000x4000 if desired
cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.CROP.TYPE = "absolute"
cfg.INPUT.CROP.SIZE = [2500, 2500]  # crop size (height, width)

# Set the output directory for model checkpoints and logs
cfg.OUTPUT_DIR = "./output_inria"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Initialize the trainer with the above configuration
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()



## Validate Model on Validation set ##


In [ ]:
# Validate the model on the validation set
evaluator = COCOEvaluator("inria_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "inria_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

## Randomly select test image ##

In [ ]:
# Change to you path for test images
test_images_dir = r"C:\Users\itagl\Downloads\Pics\Pics" 
# test_images_dir = r"C:\Users\itagl\Downloads\AerialImageDataset\test\images"


# Valid image extensions
img_extensions = [".jpg", ".jpeg", ".png", ".tif"]
all_test_files = [
    f for f in os.listdir(test_images_dir)
    if os.path.splitext(f)[1].lower() in img_extensions
]

if not all_test_files:
    raise ValueError("No valid images found in test_images_dir!")

# Randomly pick one
chosen_file = random.choice(all_test_files)
test_image_path = os.path.join(test_images_dir, chosen_file)

print("Randomly chosen test image:", test_image_path)


## Perform inference using the trained model and visualize ##

In [ ]:
# Update the configuration to use the final model weights
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4   # Set the confidence threshold
cfg.TEST.DETECTIONS_PER_IMAGE = 1000

predictor = DefaultPredictor(cfg)

# Load the test image
im = cv2.imread(test_image_path)

outputs = predictor(im)
print("Bounding boxes:", outputs["instances"].pred_boxes)
print("Segmentation masks:", outputs["instances"].pred_masks)

# Visualize the prediction
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get("inria_train"), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize=(12, 12))
plt.imshow(out.get_image()[:, :, ::-1])
plt.title("Inference Result")
plt.axis("off")
plt.show()


## Prepare data generators for roof type classification ##

In [ ]:
# Define directories
train_dir = 'path/to/roof_type/train'
val_dir = 'path/to/roof_type/val'

# Image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    vertical_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)


## Build and train the roof type classification model ##

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)


## Perform obstacle detection using edge detection and color thresholding ##

In [ ]:
# Load the roof image (segmented roof area)
roof_image = cv2.imread('path/to/segmented/roof_image.jpg')

# Convert to grayscale
gray = cv2.cvtColor(roof_image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, threshold1=50, threshold2=150)

# Find contours
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on a mask
obstacle_mask = np.zeros_like(gray)
cv2.drawContours(obstacle_mask, contours, -1, 255, thickness=cv2.FILLED)

# Optional: Use color thresholding to refine obstacle detection
hsv = cv2.cvtColor(roof_image, cv2.COLOR_BGR2HSV)
lower_val = np.array([0, 0, 0])
upper_val = np.array([180, 255, 30])  # Adjust values based on obstacle colors
color_mask = cv2.inRange(hsv, lower_val, upper_val)

# Combine edge and color masks
combined_mask = cv2.bitwise_or(obstacle_mask, color_mask)

# Visualize the obstacle mask
plt.figure(figsize=(12, 8))
plt.imshow(combined_mask, cmap='gray')
plt.axis('off')
plt.title('Obstacle Mask')
plt.show()


## Calculate the effective roof area excluding obstacles ##

In [ ]:
# Assume roof_mask is a binary mask of the roof area
roof_mask = cv2.imread('path/to/roof_mask.jpg', cv2.IMREAD_GRAYSCALE)
roof_area_pixels = cv2.countNonZero(roof_mask)

# Obstacle area in pixels
obstacle_area_pixels = cv2.countNonZero(combined_mask)

# Effective area in pixels
effective_area_pixels = roof_area_pixels - obstacle_area_pixels

# Spatial resolution in meters per pixel (you need to define this based on your data)
resolution = 0.1  # Example: each pixel represents 0.1 meters

# Convert to square meters
effective_area_m2 = effective_area_pixels * (resolution ** 2)

print(f"Effective roof area available for solar panels: {effective_area_m2:.2f} square meters")


## Detect roof faces and calculate orientation angles ##

In [ ]:
# Load the roof image
roof_image = cv2.imread('path/to/roof_image.jpg')
gray = cv2.cvtColor(roof_image, cv2.COLOR_BGR2GRAY)

# Apply edge detection
edges = cv2.Canny(gray, threshold1=50, threshold2=150)

# Use Hough Line Transform to detect lines
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)

# Create an empty image to draw lines
line_image = np.zeros_like(roof_image)

# Draw lines on the image
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Overlay lines on the original image
overlay_image = cv2.addWeighted(roof_image, 0.8, line_image, 1, 0)

# Display the image with detected lines
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(overlay_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Detected Roof Faces')
plt.show()

# Placeholder for normal vector calculation
# In reality, you would need 3D coordinates of the roof faces
# For demonstration, let's assume a normal vector
normal_vector = np.array([0.5, 0.5, 0.7071])  # Example normal vector

# Function to calculate orientation
import math

def calculate_orientation(normal_vector):
    nx, ny, nz = normal_vector
    azimuth = math.degrees(math.atan2(ny, nx)) % 360
    tilt = math.degrees(math.acos(nz / np.linalg.norm(normal_vector)))
    return azimuth, tilt

azimuth_angle, tilt_angle = calculate_orientation(normal_vector)

print(f"Azimuth Angle: {azimuth_angle:.2f} degrees")
print(f"Tilt Angle: {tilt_angle:.2f} degrees")


## Determine if the roof face is suitable for solar panel installation ##

In [ ]:
if 30 <= tilt_angle <= 40:
    print("The roof face has an ideal tilt angle for solar panels.")
    suitable_area_m2 = effective_area_m2  # Assuming the entire effective area is suitable
else:
    print("The roof face does not have an ideal tilt angle for solar panels.")
    suitable_area_m2 = 0

print(f"Suitable area for solar panels: {suitable_area_m2:.2f} square meters")


## Estimate potential energy production ##

In [ ]:
# Assuming average solar irradiance and panel efficiency
solar_irradiance = 1000  # W/m^2 (average peak sun hours)
panel_efficiency = 0.17  # 17% efficiency
system_losses = 0.14     # 14% losses

# Calculate the estimated energy production
daily_energy_kWh = suitable_area_m2 * solar_irradiance * panel_efficiency * (1 - system_losses) * 4 / 1000

print(f"Estimated daily energy production: {daily_energy_kWh:.2f} kWh")
